In [5]:
import pandas as pd

In [23]:
import json

import requests

url = "https://api.companieshouse.gov.uk/search/companies?q={}"
query = "london"
api_key = "1322a9ab-8b01-451e-9ec9-84b387cfbdae"  # Fake key - insert your key here

response = requests.get(url.format(query), auth=(api_key, ""))
json_search_result = response.text
search_result = json.JSONDecoder().decode(json_search_result)
search_result

6

In [2]:
import datetime

from CompaniesHouseService import CompaniesHouseService

# Initialise variables
key = "1322a9ab-8b01-451e-9ec9-84b387cfbdae"  # Fake key - insert your key here
company_numbers_file = "CompanyNumbers.csv"
company_output_file = "CompanyDataOutput.csv"

# Read csv into pandas dataframe
df = pd.read_csv(company_numbers_file)

# Create an instance of the wrapper class
ch_api = CompaniesHouseService(key)

# Start timer
tic = datetime.datetime.now()

# Loop through rows of dataframe
for index, row in df.iterrows():
    company_number = row["Company Number"]
    ch_profile = ch_api.get_company_profile(company_number)
    df.at[index, "Company Name"] = ch_profile.get("company_name", None)

    # SIC codes are returned as a list of up to 4 numbers
    sics = ch_profile.get("sic_codes", [None])
    for i in range(0, len(sics)):
        df.at[index, f"SIC {i+1}"] = sics[i]

    print(f"Num: {row['Company Number']} | " f"Name: {df.at[index,'Company Name']}")

# End timer
toc = datetime.datetime.now()

avg_time = ((toc - tic).total_seconds()) / (len(df.index) - 1)
print(f"Average time between API calls: {avg_time:0.2f} seconds")

# Save the results to a csv file
df.to_csv(company_output_file, index=False)

Num: 03127966 | Name: CHEQUE CLEARING COMPANY LIMITED
Num: SC380228 | Name: C & J THOMSON LTD
Num: 08090951 | Name: AIREDALE PRESERVATION LIMITED
Num: 08274073 | Name: BIOLOGICAL FARMING NEWS LTD.
Num: 09120350 | Name: BUAT INDONESIA LTD.
Num: 05487913 | Name: ATLAS BUNGALOWS MANAGEMENT COMPANY LIMITED
Num: 11052744 | Name: ADZOUK LTD
Num: 08608755 | Name: CAMBERS COUNTRY STORES LIMITED
Num: 05945470 | Name: 3JC LIMITED
Num: 10621964 | Name: ASLAN GOLF AND SPORTS LIMITED
Num: 04857873 | Name: ADEPT AND AFFORDABLE MACHIN ACCOUNTING LTD
Num: FC012676 | Name: AJINOMOTO EUROPE SALES GMBH
Num: SC515156 | Name: ACHNACREEMORE LIMITED
Num: SC636062 | Name: CAFE UNIQUE DUNDEE LIMITED
Num: 11605899 | Name: BORN SURVIVOR EVENTS LIMITED
Num: 11852632 | Name: BSG PROPERTY LIMITED
Num: SC513290 | Name: ALBERT STORES LIMITED
Num: 12200440 | Name: 101 FACILITIES GROUP LIMITED
Num: 11213791 | Name: CELLAR WORKS LTD
Num: 09202675 | Name: CANTAB RESIDENTIAL LIMITED
Num: 07079128 | Name: A & M SENIOR CARE

In [8]:
from CompaniesHouseService import CompaniesHouseService

key = "1322a9ab-8b01-451e-9ec9-84b387cfbdae"  # Fake key - insert your key here
ch_api = CompaniesHouseService(key)
ch_api.get_company_profile("04857873")

Failed with error code: 403 | Reason: Forbidden


{}

In [113]:
import pandas as pd

df = pd.read_csv(
    "BigFile.csv", usecols=[0, 1, 4, 5, 6, 9, 11, 14, 26, 27, 28, 29, 32], header=0
)
MK_light = df[df["RegAddress.PostTown"] == "MILTON KEYNES"]
MK_light = MK_light.rename(
    columns={
        " CompanyNumber": "CompanyNumber",
        " RegAddress.AddressLine2": "RegAddress.AddressLine2",
    }
)
MK_light.to_csv("CompaniesMK.csv", header=True)

# **Geocoding**

In [184]:
"	"

'\t'

In [229]:
import http.client
import urllib.parse

conn = http.client.HTTPConnection("api.positionstack.com")
work = "230 C/O KEENS SHAY KEENS MK, UPPER FIFTH STREET, MK9 2HR"
dont_work = "3 BLUNDELLS ROAD BRADVILLE \n MILTON KEYNES \n MK13 7HA"

params = urllib.parse.urlencode(
    {
        "access_key": "f51a70304f0a9f959365c92f8cbc8c2d",
        "query": dont_work,
        "region": "Milton Keynes",
        "limit": 1,
    }
)

conn.request("GET", "/v1/forward?{}".format(params))

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"data":[{"latitude":52.060143,"longitude":-0.7815,"type":"neighbourhood","name":"Bradville","number":null,"postal_code":null,"street":null,"confidence":0.6,"region":"Buckinghamshire","region_code":null,"county":"Milton Keynes","locality":"Milton Keynes","administrative_area":"Central Milton Keynes","neighbourhood":"Bradville","country":"United Kingdom","country_code":"GBR","continent":"Europe","label":"Bradville, Milton Keynes, England, United Kingdom","map_url":"https:\/\/map.positionstack.com\/export\/embed.html?bbox=-0.7865,52.055143,-0.7765,52.065143\u0026layer=mapnik\u0026marker=52.060143,-0.7815"}]}


In [91]:
import json

data = json.loads(data.decode("utf-8"))
lat = data["data"][0]["latitude"]
lon = data["data"][0]["longitude"]

In [270]:
conn = http.client.HTTPConnection("api.positionstack.com")


output = []
cpt = 0
i = 0
for index, b in MK_light.iterrows():
    i += 1
    if i > 20831:
        query = (
            str(b["RegAddress.AddressLine1"])
            + " , "
            + str(b[" RegAddress.AddressLine2"])
            + " , MILTON KEYNES , "
            + str(b["RegAddress.PostCode"])
        )
        params = urllib.parse.urlencode(
            {
                "access_key": "f51a70304f0a9f959365c92f8cbc8c2d",
                "query": query,
                "region": "Milton Keynes",
                "limit": 1,
            }
        )
        print(query)

        conn.request("GET", "/v1/forward?{}".format(params))

        res = conn.getresponse()
        data = res.read()
        data = json.loads(data.decode("utf-8"))
        output.append(data)
        print(index)
        cpt += 2
        print(cpt)

CHALLENGE HOUSE SHERWOOD DRIVE , BLETCHLEY , MILTON KEYNES , MK3 6DP
4526757
2
CHALLENGE HOUSE SHERWOOD DRIVE , BLETCHLEY , MILTON KEYNES , MK3 6DP
4526758
4
8 MOERAN CLOSE , BROWNS WOOD , MILTON KEYNES , MK7 8DT
4526865
6
35 WIMBLEDON PLACE , BRADWELL COMMON , MILTON KEYNES , MK13 8BH
4527432
8
7 AURELIA CLOSE , FAIRFIELDS , MILTON KEYNES , MK11 4AJ
4527948
10
9 HIGH STREET , WOBURN SANDS , MILTON KEYNES , MK17 8RF
4528180
12
19 DIAMOND COURT OPAL DRIVE , FOX MILNE , MILTON KEYNES , MK15 0DU
4528304
14
COSGROVE BARN, 3 MILL ROAD , HAVERSHAM , MILTON KEYNES , MK19 7DL
4528372
16
SUITE 141, MILTON KEYNES BUSINESS CENTRE FOXHUNTER DRIVE , LINFORD WOOD , MILTON KEYNES , MK14 6GD
4528500
18
82 DUNCOMBE STREET, , BLETCHLEY, , MILTON KEYNES , MK2 2LY
4528706
20
REGUS HOUSE, OFFICE 207C FAIRBOURNE DRIVE , ATTERBURY , MILTON KEYNES , MK10 9RG
4528846
22
MIDSUMMER COURT , 314 MIDSUMMER BOULEVARD , MILTON KEYNES , MK9 2UB
4528866
24
43 STONECROP PLACE , CONNIBURROW , MILTON KEYNES , MK14 7AW
452

4551874
220
6 SELBY GROVE , SHENLEY CHURCH END , MILTON KEYNES , MK5 6BW
4552054
222
12 TUTBURY LANE , KINGSMEAD , MILTON KEYNES , MK4 4LX
4552121
224
1 SHEELIN GROVE , BLETCHLEY , MILTON KEYNES , MK2 3PG
4552474
226
77 ALSTON DRIVE , BRADWELL ABBEY , MILTON KEYNES , MK13 9HG
4552510
228
UNIT 63 GARAMONDE DRIVE , WYMBUSH , MILTON KEYNES , MK8 8DD
4552640
230
13 BRANTWOOD CLOSE , WESTCROFT , MILTON KEYNES , MK4 4FP
4553076
232
19 STONELEIGH COURT , WESTCROFT , MILTON KEYNES , MK4 4BS
4553091
234
29 NORTH ELEVENTH STREET , nan , MILTON KEYNES , MK9 3BU
4553215
236
THE STABLE YARD VICARAGE ROAD , STONY STRATFORD , MILTON KEYNES , MK11 1BN
4553554
238
SUITE 8 548-550 ELDER HOUSE , ELDER GATE , MILTON KEYNES , MK9 1LR
4553700
240
THE DENE ASPLEY HILL , WOBURN SANDS , MILTON KEYNES , MK17 8NN
4554064
242
9 HARLOW CRESCENT , OXLEY PARK , MILTON KEYNES , MK4 4EP
4554238
244
26 PRIMROSE ROAD , BRADWELL VILLAGE , MILTON KEYNES , MK13 9AT
4554336
246
11 COGGESHALL GROVE , WAVENDON GATE , MILTON K

4572780
446
1ST FLOOR 181 QUEENSWAY , BLETCHLEY , MILTON KEYNES , MK2 2DZ
4572863
448
206 REGENCY COURT , UPPER FIFTH STREET , MILTON KEYNES , MK9 2HR
4573078
450
3 OAKHILL ROAD , SHENLEY CHURCH END , MILTON KEYNES , MK5 6AR
4573086
452
33 BLYTHEBRIDGE , BROUGHTON , MILTON KEYNES , MK10 7AY
4573284
454
5 WEDGEWOOD CLOSE , WAVENDON , MILTON KEYNES , MK17 8YF
4573391
456
31 FIRST AVENUE , BLETCHLEY , MILTON KEYNES , MK1 1DX
4573405
458
166 DOWNS BARN BOULEVARD , DOWNS BARN , MILTON KEYNES , MK14 7QQ
4574147
460
9 VERMONT PLACE , nan , MILTON KEYNES , MK15 8JA
4574558
462
UNIT 16 TWIZEL CLOSE , STONEBRIDGE , MILTON KEYNES , MK13 0DX
4574718
464
7 DEANSHANGER ROAD , WICKEN , MILTON KEYNES , MK19 6BS
4575003
466
C/O HILLIER HOPKINS LLP , 249 SILBURY BOULEVARD , MILTON KEYNES , MK9 1NA
4575130
468
TWO MILE ASH COMMUNITY CENTRE , THE HIGH STREET, TWO MILE ASH , MILTON KEYNES , MK8 8LH
4575642
470
72-74 THE HIGHSTREET , TWO MILE ASH , MILTON KEYNES , MK8 8HD
4575643
472
C/O IFTL, FAIRFIELDS PR

4590468
668
42 VERYAN PLACE , FISHERMEAD , MILTON KEYNES , MK6 2SR
4590578
670
44 DOUGLAS PLACE , OLDBROOK , MILTON KEYNES , MK6 2XG
4592026
672
88 SAKURA WALK , nan , MILTON KEYNES , MK15 9EH
4592671
674
2 GRANGE COURT , WOLVERTON MILL , MILTON KEYNES , MK12 5NE
4592884
676
77 ALSTON DRIVE , BRADWELL ABBEY INDUSTRIAL ESTATE , MILTON KEYNES , MK13 9HG
4593228
678
5 HERTFORD PLACE , BLETCHLEY , MILTON KEYNES , MK3 7HE
4594069
680
25 CELINA CLOSE , BLETCHLEY , MILTON KEYNES , MK2 3LS
4594251
682
15 VICTORIA ROAD , FENNY STRATFORD , MILTON KEYNES , MK2 2NG
4595175
684
UNIT 16 DENBIGH HALL INDUSTRIAL ESTATE, DENBIGH HALL , BLETCHLEY , MILTON KEYNES , MK3 7QT
4596475
686
6 CHAPTER HOUSE , COFFEE HALL , MILTON KEYNES , MK6 5EE
4597560
688
111 DOWNLAND , TWO MILE ASH , MILTON KEYNES , MK8 8JQ
4598456
690
34A STRATFORD ROAD , WOLVERTON , MILTON KEYNES , MK12 5LW
4599038
692
17 NICHOLSON GROVE , GRANGE FARM , MILTON KEYNES , MK8 0NF
4599187
694
29 GODWIN CLOSE , WAVENDON GATE , MILTON KEYNES , 

4615671
888
UNIT 16 WALKER AVENUE STRATFORD OFFICE VILLAGE , WOLVERTON MILL , MILTON KEYNES , MK12 5TW
4615750
890
5 CULMSTOCK CLOSE , EMERSON VALLEY , MILTON KEYNES , MK4 2BH
4616756
892
14 GREENELL CLOSE , nan , MILTON KEYNES , MK19 6UX
4616771
894
43 PORTLAND DRIVE , WILLEN , MILTON KEYNES , MK15 9HU
4617223
896
UNO ESTATE, HARBEN HOUSE TICKFORD STREET , NEWPORT PAGNELL , MILTON KEYNES , MK16 9EY
4617330
898
1 NEWTON COURT KELVIN DRIVE , KNOWLHILL , MILTON KEYNES , MK5 8NH
4617433
900
1 HATHAWAY COURT , nan , MILTON KEYNES , MK8 0LG
4617476
902
8 CROWBOROUGH LANE , KENTS HILL , MILTON KEYNES , MK7 6HF
4617818
904
15 CHAMPFLOWER , FURZTON , MILTON KEYNES , MK4 1LD
4618028
906
77 SINATRA DRIVE , OXLEY PARK , MILTON KEYNES , MK4 4JH
4618403
908
4 WARREN YARD , WOLVERTON MILL , MILTON KEYNES , MK12 5NW
4618564
910
4 WARREN YARD , WOLVERTON MILL , MILTON KEYNES , MK12 5NW
4618565
912
4 WARREN YARD , WOLVERTON MILL , MILTON KEYNES , MK12 5NW
4618617
914
MKTAX TECHNOLOGY HOUSE , 151SILBURY

...

4805633
2628
21 HAMPSON CLOSE , BRADWELL , MILTON KEYNES , MK13 9DG
4805830
2630
C/O HILLIER HOPKINS LLP , 249 SILBURY BOULEVARD , MILTON KEYNES , MK9 1NA
4805855
2632
C/O HILLIER HOPKINS LLP , 249 SILBURY BOULEVARD , MILTON KEYNES , MK9 1NA
4805860
2634
314 MIDSUMMER BOULEVARD , nan , MILTON KEYNES , MK9 2UB
4806364
2636
6 SWAN HUNTER CLOSE , nan , MILTON KEYNES , MK10 7GF
4806623
2638
6 SWAN HUNTER CLOSE , nan , MILTON KEYNES , MK10 7GF
4806624
2640
6 SWAN HUNTER CLOSE , BROOKLANDS , MILTON KEYNES , MK10 7GF
4806625
2642
BRICK HOUSE 150A STATION ROAD , WOBURN SANDS , MILTON KEYNES , MK17 8SG
4806764
2644
222 WOLVERTON ROAD , BLAKELANDS , MILTON KEYNES , MK14 5AB
4807134
2646
PO BOX , 7608 , MILTON KEYNES , MK11 9JG
4807387
2648
110 HIGH STREET , STONY STATFORD , MILTON KEYNES , MK11 1AH
4807732
2650
48A AYLESBURY STREET , FENNY STRATFORD , MILTON KEYNES , MK2 2BA
4809027
2652
48A AYLESBURY STREET , FENNY STRATFORD , MILTON KEYNES , MK2 2BA
4809028
2654
1 RADIAN COURT , KNOWLHILL , MI

4825547
2846
12 DUKE STREET , ASPLEY GUISE , MILTON KEYNES , MK17 8EF
4826761
2848
WESTFALIA HOUSE OLD WOLVERTON ROAD , OLD WOLVERTON , MILTON KEYNES , MK12 5PY
4826959
2850
5 GLYN SQUARE , WOLVERTON , MILTON KEYNES , MK12 5JQ
4827715
2852
BUSHFIELD SCHOOL MOON STREET , WOLVERTON , MILTON KEYNES , MK12 5JG
4827727
2854
WOLVERTON SWIMMING AND FITNESS CENTRE ADDINGTON AVENUE , WOLVERTON , MILTON KEYNES , MK12 5GH
4827728
2856
WOLVERTON SWIMMING & FITNESS CENTRE ADDINGTON AVENUE , WOLVERTON , MILTON KEYNES , MK12 5GH
4827729
2858
25 AYLESBURY STREET , WOLVERTON , MILTON KEYNES , MK12 5HY
4827730
2860
22 THE SQUARE , WOLVERTON , MILTON KEYNES , MK12 5DG
4827734
2862
4 WATERMILL LANE , WOLVERTON MILL , MILTON KEYNES , MK12 5PR
4827736
2864
171 BEADLEMEAD , NETHERFIELD , MILTON KEYNES , MK6 4HU
4827855
2866
AURORA HOUSE DELTIC AVENUE , ROOKSLEY , MILTON KEYNES , MK13 8LW
4827859
2868
MOORGATE HOUSE , 201 SILBURY BOULEVARD , MILTON KEYNES , MK9 1LZ
4827894
2870
20 BURNERS LANE , KILN FARM , M

4851409
3062
26 SAKURA WALK , WILLEN PARK , MILTON KEYNES , MK15 9EH
4851766
3064
C/O HILLIER HOPKINS LLP , 249 SILBURY BOULEVARD , MILTON KEYNES , MK9 1NA
4852111
3066
18 LONDON ROAD , STONY STRATFORD , MILTON KEYNES , MK11 1JL
4852118
3068
16 STRATFORD ROAD , WOLVERTON , MILTON KEYNES , MK12 5LJ
4852775
3070
UNITS A, B & C HEYFIELD BUSINESS PARK , ASPLEY GUISE , MILTON KEYNES , MK17 8HS
4853031
3072
THE STABLE YARD VICARAGE ROAD , STONY STRATFORD , MILTON KEYNES , MK11 1BN
4853319
3074
19 HUNSDON CLOSE , SANTONBURY FIELDS , MILTON KEYNES , MK14 6TB
4853352
3076
27 GUNVER LANE , TATTENHOE , MILTON KEYNES , MK4 3DB
4853744
3078
29  STRATFORD OFFICE VILLAGE WALKER AVENUE , WOLVERTON MILL , MILTON KEYNES , MK12 5TW
4855766
3080
19 NEWBOLT CLOSE , NEWPORT PAGNELL , MILTON KEYNES , MK16 8ND
4856134
3082
HOUSE OF THE OLD WAYS, 7 TANSMAN LANE , OLD FARM PARK , MILTON KEYNES , MK7 8EP
4856200
3084
UNIT 5 CLARKE ROAD , MOUNT FARM INDUSTRIAL ESTATE , MILTON KEYNES , MK1 1LG
4856717
3086
15 TOWC

4879748
3284
368 FEN STREET , BROOKLANDS , MILTON KEYNES , MK10 7JR
4880052
3286
YORK HOUSE CENTRE , LONDON ROAD STONY STRATFORD , MILTON KEYNES , MK11 1JQ
4880699
3288
SHROB LODGE , OLD STRATFORD , MILTON KEYNES , MK19 6BA
4881258
3290
31 WALKER AVENUE , WOLVERTON MILL EAST , MILTON KEYNES , MK12 5TW
4881319
3292
AURORA HOUSE DELTIC AVENUE , ROOKSLEY , MILTON KEYNES , MK13 8LW
4883623
3294
GPG HOUSE UNIT 8 WALKER AVENUE , WOLVERTON , MILTON KEYNES , MK12 5TW
4883690
3296
23 HEATHFIELD , STACEY BUSHES , MILTON KEYNES , MK12 6HP
4883828
3298
9 MONTAGUE CRESCENT , nan , MILTON KEYNES , MK10 7LN
4883948
3300
UNIT 47 BARTON ROAD , BLETCHLEY , MILTON KEYNES , MK2 3HU
4884848
3302
FLAT 7 55 MILLWARD DRIVE , BLETCHLEY , MILTON KEYNES , MK2 2DB
4885017
3304
61 HARLEY DRIVE , WALTON , MILTON KEYNES , MK7 7EX
4885118
3306
1 RADIAN COURT , KNOWLHILL , MILTON KEYNES , MK5 8PJ
4885280
3308
13 FARTHING GROVE FARTHING GROVE , NETHERFIELD , MILTON KEYNES , MK6 4JH
4885615
3310
UNIT W47 BLETCHLEY BUSIN

4908185
3502
33 GLOUCESTER ROAD , WOLVERTON , MILTON KEYNES , MK12 5EA
4908209
3504
UNIT 13 DRAKES MEWS BUSINESS CENTRE , CROWNHILL , MILTON KEYNES , MK8 0ER
4908307
3506
43 AYLESBURY STREET , BLETCHLEY , MILTON KEYNES , MK2 2BQ
4908513
3508
G297 DRAKES MEWS BUSINESS CENTRE , CROWNHILL , MILTON KEYNES , MK8 0ER
4909564
3510
50 KELLING WAY , BROUGHTON , MILTON KEYNES , MK10 9NW
4909634
3512
1 SISSINGHURST DRIVE , WESTCROFT , MILTON KEYNES , MK4 4FZ
4909654
3514
C/O HILLIER HOPKINS LLP , 249 SILBURY BOULEVARD , MILTON KEYNES , MK9 1NA
4909674
3516
4 HATCHLANDS , GREAT HOLM , MILTON KEYNES , MK8 9DN
4909736
3518
58B HIGH STREET , STONY STRATFORD , MILTON KEYNES , MK11 1AQ
4909768
3520
58B HIGH STREET , STONY STRATFORD , MILTON KEYNES , MK11 1AQ
4909787
3522
35 BURGH DRIVE , NEWTON LEYS , MILTON KEYNES , MK3 5RJ
4909803
3524
3 CLEGG SQUARE , SHENLEY LODGE , MILTON KEYNES , MK5 7HG
4909853
3526
37 WINDMILL HILL DRIVE , BLETCHLEY , MILTON KEYNES , MK3 7RE
4909972
3528
10 ORMONDE , STANTONBUR

In [1]:
MK_light

NameError: name 'MK_light' is not defined

In [269]:
41662 // 2

20831

In [274]:
len(output)

1816

In [289]:
output[44]["data"]

[[]]

In [292]:
output[1]
lats = []
lons = []
for i in range(len(output)):
    try:
        lats.append(output[i]["data"][0]["latitude"])
        lons.append(output[i]["data"][0]["longitude"])
    except:
        # print(output[1]["data"][i]['latitude'])
        print(output[i])

{'data': [[]]}
{'data': [[]]}
{'data': []}
{'data': [[]]}
{'data': [[]]}
{'data': []}
{'data': [[]]}
{'data': [[]]}
{'data': []}
{'data': []}
{'data': [[]]}
{'data': [[]]}
{'data': [[]]}
{'data': [[]]}
{'data': []}
{'data': [[]]}
{'data': []}
{'data': [[]]}
{'data': [[]]}
{'data': [[]]}
{'data': []}
{'data': [[]]}
{'data': [[]]}
{'data': [[]]}
{'data': [[]]}
{'data': []}
{'data': [[]]}
{'data': [[]]}
{'data': [[]]}
{'data': [[]]}
{'data': [[]]}
{'data': [[]]}
{'data': [[]]}
{'data': [[]]}
{'data': []}
{'data': []}
{'data': [[]]}
{'data': [[]]}
{'data': [[]]}
{'data': [[]]}
{'data': [[]]}
{'data': [[]]}
{'data': [[]]}
{'data': []}
{'data': [[]]}
{'data': []}
{'data': [[]]}
{'data': [[]]}
{'data': []}
{'data': []}
{'data': [[]]}
{'data': [[]]}
{'data': [[]]}
{'data': [[]]}
{'data': [[]]}
{'data': []}
{'data': [[]]}
{'data': [[]]}
{'data': [[]]}
{'data': [[]]}
{'data': [[]]}
{'data': []}
{'data': [[]]}
{'data': [[]]}
{'data': [[]]}
{'data': [[]]}
{'data': [[]]}
{'data': [[]]}
{'data': [[]

# **Geopandas**

In [297]:
df = pd.DataFrame(list(zip(lats, lons)), columns=["latitude", "longitude"])

In [298]:
df

,latitude,longitude
0,52.001588,-0.740203
1,52.014831,-0.686313
2,52.042038,-0.767004
3,52.042555,-0.833628
4,52.016090,-0.650147
...,...,...
1484,52.001653,-0.774793
1485,52.046983,-0.705743
1486,52.046983,-0.705743
1487,52.046983,-0.705743


In [ ]:
import plotly.express as px

fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", zoom=10)
fig.update_layout(mapbox_style="open-street-map")

In [306]:
x = df[:10]

In [308]:
fig.show()

In [312]:
import os

os.listdir()

['CompaniesHouseService.py',
 'Untitled.ipynb',
 'CompaniesHouseMK.ipynb',
 'CompanyDataOutput.csv',
 'CompaniesMK.csv',
 '__pycache__',
 '.ipynb_checkpoints',
 'BigFile.csv',
 'CompanySICs.py',
 'CompanyNumbers.csv',
 'OutputFile.csv']

In [6]:
df = pd.read_csv("CompaniesMK.csv", index_col=0)

In [7]:
df[df["CompanyStatus"] == "Active"]

,CompanyName,CompanyNumber,RegAddress.AddressLine1,RegAddress.AddressLine2,RegAddress.PostTown,RegAddress.PostCode,CompanyStatus,IncorporationDate,SICCode.SicText_1,SICCode.SicText_2,SICCode.SicText_3,SICCode.SicText_4,URI
596,(UK) SWEET HOMES LTD,12060582,3 BLUNDELLS ROAD,BRADVILLE,MILTON KEYNES,MK13 7HA,Active,19/06/2019,68100 - Buying and selling of own real estate,68209 - Other letting and operating of own or ...,68310 - Real estate agencies,68320 - Management of real estate on a fee or ...,http://business.data.gov.uk/id/company/12060582
1125,00947912 LIMITED,00947912,PITFIELD,KILN FARM,MILTON KEYNES,MK11 3DR,Active,13/02/1969,33140 - Repair of electrical equipment,NaN,NaN,NaN,http://business.data.gov.uk/id/company/00947912
2152,0K - OFFICIALLY THE WORLD'S COOLEST BRAND LIMITED,09141489,1 HAYNES CLOSE,BOW BRICKHILL,MILTON KEYNES,MK17 9JQ,Active,22/07/2014,99999 - Dormant Company,NaN,NaN,NaN,http://business.data.gov.uk/id/company/09141489
2405,1 BROOKLANDS COURT LIMITED,11081797,BAIRD HOUSE SEEBECK PLACE,KNOWLHILL,MILTON KEYNES,MK5 8FR,Active,24/11/2017,68209 - Other letting and operating of own or ...,NaN,NaN,NaN,http://business.data.gov.uk/id/company/11081797
2923,1 MK TAXI LIMITED,12576511,71 STRATFORD ROAD,WOLVERTON,MILTON KEYNES,MK12 5LT,Active,28/04/2020,49320 - Taxi operation,NaN,NaN,NaN,http://business.data.gov.uk/id/company/12576511
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4918040,ZURI CONSULTING LIMITED,12867637,2 KINDERFIELD CLOSE,EMERSON VALLEY,MILTON KEYNES,MK4 2AW,Active,09/09/2020,82990 - Other business support service activit...,NaN,NaN,NaN,http://business.data.gov.uk/id/company/12867637
4918047,ZURI HEALTH CARE LTD,10948428,7 TORRE CLOSE,BLETCHLEY,MILTON KEYNES,MK3 6NG,Active,06/09/2017,86900 - Other human health activities,NaN,NaN,NaN,http://business.data.gov.uk/id/company/10948428
4918818,ZYGOLOGY SCIENTIFIC LIMITED,13228300,ARTEMIS HOUSE,"4A BRAMLEY RD, MOUNT FARM",MILTON KEYNES,MK1 1PT,Active,26/02/2021,46660 - Wholesale of other office machinery an...,NaN,NaN,NaN,http://business.data.gov.uk/id/company/13228300
4918819,ZYGOLOGY SYSTEMS LIMITED,03696606,ARTEMIS HOUSE 4A BRAMLEY ROAD,MOUNT FARM,MILTON KEYNES,MK1 1PT,Active,18/01/1999,46660 - Wholesale of other office machinery an...,NaN,NaN,NaN,http://business.data.gov.uk/id/company/03696606


In [46]:
import pandas as pd

data_MK = pd.read_csv(
    "/Users/cordiez/Documents/CompaniesHouse/OutputFiles/gmaps_details.csv", index_col=0
)

In [47]:
len(data_MK[~data_MK["details_output_from_candidate_request"].isna()])

15831

In [50]:
import pandas as pd

data_MK = pd.read_csv(
    "/Users/cordiez/Documents/CompaniesHouse/OutputFiles/MK_geocoded.csv", index_col=0
)

import plotly.express as px

fig = px.scatter_mapbox(data_MK, lat="Lat", lon="Lon", zoom=10)
fig.update_layout(mapbox_style="open-street-map")